In [1]:
import warnings
warnings.filterwarnings('ignore')

import time
import os
import re
import pandas as pd
import random

from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.lines as mlines
from   matplotlib.lines import Line2D
from   matplotlib.colors import ListedColormap
import matplotlib.ticker as tk

import numpy as np
from numpy import ndarray

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from scipy.stats import truncnorm

import multiprocessing
import pickle
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import (Input, Dropout, LSTM, Reshape, LeakyReLU,
                          Concatenate, ReLU, Flatten, Dense, Embedding,
                          BatchNormalization, Activation, SpatialDropout1D,
                          Conv2D, MaxPooling2D, UpSampling2D, Lambda)
from tensorflow.keras.models     import Model, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses     import mse, binary_crossentropy
import tensorflow.keras.backend as K
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.metrics import  mean_squared_error as mse_keras
from tensorflow.keras.backend import argmax as argmax
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import one_hot
from tensorflow.keras.models import Sequential 

from tensorflow.keras.utils import  to_categorical
from tensorflow import random as randomtf

from IPython.display import clear_output


from progressbar import ProgressBar
import seaborn as sns

from chainer_chemistry.dataset.preprocessors import GGNNPreprocessor, construct_atomic_number_array
preprocessor = GGNNPreprocessor()
from rdkit import rdBase
rdBase.DisableLog('rdApp.error')
from rdkit import Chem

import ntpath
from scipy.stats import truncnorm

randomtf.set_seed(10)
os.environ['PYTHONHASHSEED'] = '10'
np.random.seed(420)
random.seed(123450)
from progressbar import ProgressBar

2023-06-13 23:03:45.272913: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-13 23:03:45.343751: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-06-13 23:03:45.678150: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda-11.2/lib64:
2023-06-13 23:03:45.678197: W tensorflow/compile

In [3]:
with open('./image_train.pickle', 'rb') as f:
    X_smiles_train0, SMILES_train0, y_train00 = pickle.load(f)
    
with open('./image_test.pickle', 'rb') as f:
    X_smiles_val0, SMILES_val0, y_val00 = pickle.load(f)

In [26]:
X_smiles_train1 = X_smiles_train0[0:20000]
X_smiles_train2 = X_smiles_train0[20000:40000]
X_smiles_train3 = X_smiles_train0[40000:60000]
X_smiles_train4 = X_smiles_train0[60000:80000]
X_smiles_train5 = X_smiles_train0[80000:100000]
X_smiles_train6 = X_smiles_train0[100000:]

SMILES_train1 = SMILES_train0[0:20000]
SMILES_train2 = SMILES_train0[20000:40000]
SMILES_train3 = SMILES_train0[40000:60000]
SMILES_train4 = SMILES_train0[60000:80000]
SMILES_train5 = SMILES_train0[80000:100000]
SMILES_train6 = SMILES_train0[100000:]

y_train1 = y_train00[0:20000]
y_train2 = y_train00[20000:40000]
y_train3 = y_train00[40000:60000]
y_train4 = y_train00[60000:80000]
y_train5 = y_train00[80000:100000]
y_train6 = y_train00[100000:]

X_smiles_val1 = X_smiles_val0[0:15000]
X_smiles_val2 = X_smiles_val0[15000:]

SMILES_val1 = SMILES_val0[0:15000]
SMILES_val2 = SMILES_val0[15000:]

y_val1 = y_val00[0:15000]
y_val2 = y_val00[15000:]

In [28]:
SMILES_train6.shape

(6101,)

In [29]:
with open('./image_train1.pickle', 'wb') as f:
    pickle.dump((X_smiles_train1, SMILES_train1, y_train1), f)
with open('./image_train2.pickle', 'wb') as f:
    pickle.dump((X_smiles_train2, SMILES_train2, y_train2), f)
with open('./image_train3.pickle', 'wb') as f:
    pickle.dump((X_smiles_train3, SMILES_train3, y_train3), f)
with open('./image_train4.pickle', 'wb') as f:
    pickle.dump((X_smiles_train4, SMILES_train4, y_train4), f)
with open('./image_train5.pickle', 'wb') as f:
    pickle.dump((X_smiles_train5, SMILES_train5, y_train5), f)
with open('./image_train6.pickle', 'wb') as f:
    pickle.dump((X_smiles_train6, SMILES_train6, y_train6), f)

In [30]:
with open('./image_val1.pickle', 'wb') as f:
    pickle.dump((X_smiles_val1, SMILES_val1, y_val1), f)
with open('./image_val2.pickle', 'wb') as f:
    pickle.dump((X_smiles_val2, SMILES_val2, y_val2), f)